In [11]:
# pip install psgdemos

In [1]:
import numpy as np
import PySimpleGUI as sg
from PIL import Image, ImageTk

In [2]:
def on_hold_grid(MAX_ROWS=38,MAX_COL=23):
    """Creates an array of the coordiantes of holds that exist on kilterboard app / are clickable"""
    
    main_grid = np.array([[(row,col) for row in range(0,MAX_ROWS,2)] for col in range(1,MAX_COL,2)]).reshape(-1,2)
    foot_right_grid = np.array([[(x,y) for x in range(5,MAX_ROWS-1,4)]for y in range(2,MAX_COL,4)]).reshape(-1,2)
    foot_left_grid = np.array([[(x,y) for x in range(7,MAX_ROWS-3,4)]for y in range(0,MAX_COL,4)]).reshape(-1,2)
    bottom_foot_grid = np.array([[(37,y) for y in range(0,MAX_COL,2)]]).reshape(-1,2)
    
    on_holds = np.concatenate((main_grid,foot_right_grid,foot_left_grid,bottom_foot_grid),axis=0)
    
    return on_holds

In [3]:

def Kilter():
    sg.change_look_and_feel('Dark Blue')
    MAX_ROWS = 38
    MAX_COL = 23
    
    grid = [['-' for col in range(MAX_COL)] for row in range(MAX_ROWS)]
    on_holds = on_hold_grid()
    
    sg.set_options(button_element_size=(12, 1),
               element_padding=(0, 0),
               auto_size_buttons=False,
               border_width=0,
               button_color=('grey90','grey30')
               )
    
    # Top Layer 
    layout =   [
               [sg.Text('Total :', text_color='white',  size=(10,0), key='Total', font='Default 20')],
               [sg.Text('Hand :', text_color='cyan', size=(10,1), key='Hand', font='Default 20')],
               [sg.Text('Foot :', text_color='orange', size=(10,1), key='Foot', font='Default 20')],
               [sg.Text('Start :', text_color='green2', size=(10,1), key='Start', font='Default 20')],
               [sg.Text('Finish :', text_color='magenta2', size=(10,1), key='Finish', font='Default 20')],
               [sg.Column([[sg.Button('Reset')]], element_justification='right', expand_x=True)]]
    # Grid of buttons
    layout +=  [[sg.Button(str(''), disabled = True, button_color=('black','grey70'), size=(2, 1), pad=(0,0), border_width=0, key=(row,col)) for col in range(MAX_COL)] for row in range(MAX_ROWS)]
    
    layout += [[sg.Column([[sg.Button('Exit')]], element_justification='center', expand_x=True)]]
    window = sg.Window('Kilterboard Lookup', layout).Finalize()
    
    
    [window[(x,y)].update('*', disabled = False, button_color=('black','grey70')) for x,y in on_holds]
    
    hand_location = []
    foot_location = []
    start_location = []
    finish_location = []

    while True:         # The Event Loop
        event, values = window.read()
#         print(event)
        if event in (None, 'Exit'):
#             print(grid)
            break
            
        if event in (None, 'Reset'):
            grid = [['-' for col in range(MAX_COL)] for row in range(MAX_ROWS)]
            [window[(x,y)].update('*', disabled = False, button_color=('black','grey70')) for x,y in on_holds]
            hand_location = []
            foot_location = []
            start_location = []
            finish_location = []
            window['Total'].update(f"Total : {len(hand_location)+len(foot_location)+len(start_location)+len(finish_location)}")
            window['Hand'].update(f"Hand : {len(hand_location)}")
            window['Foot'].update(f"Foot : {len(foot_location)}")
            window['Start'].update(f"Start : {len(start_location)}")
            window['Finish'].update(f"Finish : {len(finish_location)}")
            continue
            
        hand_length = len(hand_location)
        foot_length = len(foot_location)
        start_length = len(start_location)
        finish_length = len(finish_location)
        total_length = hand_length + foot_length + start_length + finish_length
        
        clicked_button = grid[event[0]][event[1]]
        
        if clicked_button == '-':
            
            if not [event[0],event[1]] in on_holds.tolist():
                print("not a hold")
                continue
            
            if total_length >=35:
                sg.popup('Max Hold Limit Reached!','Please remove or change existing holds', title = 'Hold Limit Reached', keep_on_top=True)
                continue
            else:
                #Turn to hand
                grid[event[0]][event[1]] = 'Hand'
                window[event].update('Hand', button_color=('white','cyan'))
                hand_location.append(event)
            
        elif clicked_button == 'Hand':
            #Turn to foot
            hand_location.remove(event)
            grid[event[0]][event[1]] = 'Foot'
            window[event].update('Foot', button_color=('white','orange'))
            foot_location.append(event)
            
        elif clicked_button == 'Foot':
            foot_location.remove(event)
            if start_length >= 2:
                if finish_length >=2:
                    #Skip Start & Finish, turn to blank
                    grid[event[0]][event[1]] = '-'
                    window[event].update('*', button_color=('black','grey70'))
                else: 
                    #Skip start, turn to finish
                    grid[event[0]][event[1]] = 'Finish'
                    window[event].update('Finish', button_color=('white','magenta2'))
                    finish_location.append(event)
            else:
                #Turn to start
                grid[event[0]][event[1]] = 'Start'
                window[event].update('Start', button_color=('white','green2'))
                start_location.append(event)
            
        elif clicked_button == 'Start':
            start_location.remove(event)
            if finish_length >=2:
                    #Skip Finish, turn to blank
                    grid[event[0]][event[1]] = '-'
                    window[event].update('*', button_color=('black','grey70'))
            else: 
                #Turn to finish
                grid[event[0]][event[1]] = 'Finish'
                window[event].update('Finish', button_color=('white','magenta2'))
                finish_location.append(event)
                    
        elif clicked_button == 'Finish':
            #Turn to blank
            finish_location.remove(event)
            grid[event[0]][event[1]] = '-'
            window[event].update('*', button_color=('black','grey70'))
            
    
#         print(grid)
        
        window['Total'].update(f"Total : {len(hand_location)+len(foot_location)+len(start_location)+len(finish_location)}")
        window['Hand'].update(f"Hand : {len(hand_location)}")
        window['Foot'].update(f"Foot : {len(foot_location)}")
        window['Start'].update(f"Start : {len(start_location)}")
        window['Finish'].update(f"Finish : {len(finish_location)}")

    
    window.close()



In [4]:
Kilter()

In [5]:
import json
with open('hold_types.json', 'r') as f:
    hold_type_dict = json.load(f)

In [71]:
class hold(object):
    
    def __init__(self, hold_number):
        self.hold_number = hold_number
        self.hold_type = hold_type_dict[str(hold_number)]

    def set_hold(self, set_type): 
        self.set_type = set_type 
    
    def get_image(self):
        return f"{self.hold_number}{self.set_type}.jpg"
    
    def __repr__(self):
        return f"This is a {self.set_type} hold at index {self.hold_number}"
    
    def __lt__(self,other):
        return self.hold_number < other.hold_number

In [ ]:
class hand(hold):
    
    def init(self, fname, lname):
        super().init(fname, lname)\
    
    

In [72]:
hold(0) < hold(1)

True

In [73]:
my_hold = hold(0)
my_hold.hold_number

0

In [74]:
my_hold.hold_type

['Jug']

In [75]:
my_hold.set_hold('finish')
my_hold.set_type

'finish'

In [76]:
my_hold.get_image()

'0finish.jpg'

In [77]:
print(my_hold)

This is a finish hold at index 0


In [78]:
my_hold.hold_number = "This shouldn't be possible"

In [79]:
my_hold.hold_number

"This shouldn't be possible"

In [81]:
my_hold.set_hold = lambda x : "Nope"

In [82]:
my_hold.set_hold('test')

'Nope'